# Praca Domowa 4 - Wstęp do uczenia maszynowego
## Kinga Ułasik IiAD 27.04.2021

# Potrzebne importy

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from numpy import log
import category_encoders as ce
from sklearn import svm
import warnings
warnings.filterwarnings('ignore')
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Wczytanie danych

### Apartments (pakiet DALEX)

In [3]:
apartments = pd.read_csv('apartments.csv')
apartments.head()

,m2.price,construction.year,surface,floor,no.rooms,district
0,5897,1953,25,3,1,Srodmiescie
1,1818,1992,143,9,5,Bielany
2,3643,1937,56,1,2,Praga
3,3517,1995,93,7,3,Ochota
4,3013,1992,144,6,5,Mokotow


In [4]:
#zastosujemy one hot encoding dla zmienej district
#apartments = pd.get_dummies(apartments_org.district, sparse=True)
#apartments = pd.concat([apartments_org, apartments], axis=1)
#apartments = apartments.drop('district', axis=1)
#apartments.head(3)

### Fifa (pakiet DALEX)

In [2]:
fifa_org = pd.read_csv('fifa.csv')
fifa_org.head()

,nationality,overall,potential,wage_eur,value_eur,age,height_cm,weight_kg,attacking_crossing,attacking_finishing,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,Argentina,94,94,565000.0,95500000.0,32,170,72,88,95,...,75,96,33,37,26,6,11,15,14,8
1,Portugal,93,93,405000.0,58500000.0,34,187,83,84,94,...,85,95,28,32,24,7,11,15,14,11
2,Brazil,92,92,290000.0,105500000.0,27,175,68,87,87,...,90,94,27,26,29,9,9,15,15,11
3,Slovenia,91,93,125000.0,77500000.0,26,188,87,13,11,...,11,68,27,12,18,87,92,78,90,89
4,Belgium,91,91,470000.0,90000000.0,28,175,74,81,84,...,88,91,34,27,22,11,12,6,8,8


In [3]:
#w tym zbiorze danych mamy potencjalnie 4 zmienne targetowe
#postanowiłam wybrać wartość piłkarza ('value_eur')
#dropujemy kolumnę 'nationality' (tak jest zalecane w opisie datasetu)
fifa = fifa_org.drop(['nationality'], axis=1)
fifa.head(3)

,overall,potential,wage_eur,value_eur,age,height_cm,weight_kg,attacking_crossing,attacking_finishing,attacking_heading_accuracy,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,94,94,565000.0,95500000.0,32,170,72,88,95,70,...,75,96,33,37,26,6,11,15,14,8
1,93,93,405000.0,58500000.0,34,187,83,84,94,89,...,85,95,28,32,24,7,11,15,14,11
2,92,92,290000.0,105500000.0,27,175,68,87,87,62,...,90,94,27,26,29,9,9,15,15,11


In [4]:
#dla ułatwienia
fifa['value_eur'] = fifa['value_eur'].div(1000000)
fifa.head()

,overall,potential,wage_eur,value_eur,age,height_cm,weight_kg,attacking_crossing,attacking_finishing,attacking_heading_accuracy,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,94,94,565000.0,95.5,32,170,72,88,95,70,...,75,96,33,37,26,6,11,15,14,8
1,93,93,405000.0,58.5,34,187,83,84,94,89,...,85,95,28,32,24,7,11,15,14,11
2,92,92,290000.0,105.5,27,175,68,87,87,62,...,90,94,27,26,29,9,9,15,15,11
3,91,93,125000.0,77.5,26,188,87,13,11,15,...,11,68,27,12,18,87,92,78,90,89
4,91,91,470000.0,90.0,28,175,74,81,84,61,...,88,91,34,27,22,11,12,6,8,8


# Support Vector Machine

# Apartments

In [15]:
rgs = svm.SVR(kernel='rbf')
one_hot_encoder = ce.OneHotEncoder()
scaler = StandardScaler()

#zastosujemy one hot encoding dla columny district
apartments=one_hot_encoder.fit_transform(apartments)

#bez skalowania
pipe_regressor = Pipeline(
[
    ('regressor', rgs)
])

#ze skalowaniem
pipe_scaler_regressor = Pipeline(
[
    ('scaler', scaler),
    ('regressor', rgs)
])

In [16]:
X = apartments.drop('m2.price',axis=1)
y = apartments.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

### Bez skalowania

In [17]:
pipe_regressor.fit(X_train, y_train)
y_pred = pipe_regressor.predict(X_test)
print('RMSE : ' + str(mean_squared_error(y_test,y_pred, squared=False)))
print('Mean of the target variable: ' + str(y.mean()))

RMSE : 951.8243490631207
Mean of the target variable: 3487.019


###  Ze skalowaniem

In [18]:
pipe_scaler_regressor.fit(X_train, y_train)
y_pred = pipe_scaler_regressor.predict(X_test)
print('RMSE : ' + str(mean_squared_error(y_test, y_pred, squared=False)))
print('Mean of the target variable: ' + str(y.mean()))

RMSE : 929.9981400650855
Mean of the target variable: 3487.019


Zauważmy, że po skalowaniu RMSE jest zauważalnie mniejsze niż bez skalowania

### Strojenie hiperparametrów

In [10]:
rgs = svm.SVR(kernel='poly', cache_size=1024)
grid = dict(
    degree=[1,2,3,4,5,6,7,8,9,10],
    gamma=['scale','auto'],
    C=uniform(loc=0, scale=8),
    coef0=uniform(loc=0, scale=8),
    max_iter=[1,25,50,100,500,1000,10000,100000,1000000],
    )
random_search = RandomizedSearchCV(rgs, grid, random_state=0)
search = random_search.fit(X_train, y_train)
search.best_params_

{'C': 6.778013910273003,
 'coef0': 4.988509574287779,
 'degree': 3,
 'gamma': 'scale',
 'max_iter': 100000}

In [14]:
rgs = svm.SVR(kernel='poly',cache_size=1024,C = 6.778013910273003,coef0 = 4.988509574287779,degree = 3,gamma ='scale',max_iter=100000)

pipe_scaler_regressor = Pipeline(
[
    ('scaler', scaler),
    ('regressor', rgs)
])

pipe_scaler_regressor.fit(X_train, y_train)
y_pred = pipe_scaler_regressor.predict(X_test)
print('RMSE : ' + str(mean_squared_error(y_test,y_pred, squared=False)))
print('R2 squared : ' + str(r2_score(y_test,y_pred)))
print('Mean of the target variable: ' + str(y.mean()))

RMSE : 200.43520138582562
R2 squared : 0.9547354913344622
Mean of the target variable: 3487.019


Po nastrojeniu hiperparametrów RMSE jest znacznie niższe, a model działa znacznie lepiej.

Dodatkowo zostało zbadane R2, które wynosi ok 0.95 co oznacza, że regresja jest bardzo dobrze dopasowana

# Fifa

In [10]:
rgs = svm.SVR(kernel='poly')
scaler = StandardScaler()

pipe_scaler_regressor = Pipeline(
[
    ('scaler', scaler),
    ('regressor', rgs)
])
pipe_regressor = Pipeline(
[
    ('regressor', rgs)
])


In [5]:
X = fifa.drop('value_eur',axis=1)
y = fifa.iloc[:,3]
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

In [6]:
X.head()

,overall,potential,wage_eur,age,height_cm,weight_kg,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,...,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,94,94,565000.0,32,170,72,88,95,70,92,...,75,96,33,37,26,6,11,15,14,8
1,93,93,405000.0,34,187,83,84,94,89,83,...,85,95,28,32,24,7,11,15,14,11
2,92,92,290000.0,27,175,68,87,87,62,87,...,90,94,27,26,29,9,9,15,15,11
3,91,93,125000.0,26,188,87,13,11,15,43,...,11,68,27,12,18,87,92,78,90,89
4,91,91,470000.0,28,175,74,81,84,61,89,...,88,91,34,27,22,11,12,6,8,8


In [7]:
y.head()

0     95.5
1     58.5
2    105.5
3     77.5
4     90.0
Name: value_eur, dtype: float64

### Bez skalowania

In [11]:
pipe_regressor.fit(X_train, y_train)
y_pred = pipe_regressor.predict(X_test)
print('RMSE : ' + str(mean_squared_error(y_test,y_pred, squared=False)))
print('Mean of the target variable: ' + str(y.mean()))

RMSE : 7.15666674027615
Mean of the target variable: 7.473286999999999


### Ze skalowaniem

In [12]:
pipe_scaler_regressor.fit(X_train, y_train)
y_pred = pipe_scaler_regressor.predict(X_test)
print('RMSE : ' + str(mean_squared_error(y_test, y_pred, squared=False)))
print('Mean of the target variable: ' + str(y.mean()))

RMSE : 3.001659670808304
Mean of the target variable: 7.473286999999999


### Strojenie hiperparametrów

In [13]:
rgs = svm.SVR(kernel='poly', cache_size=1024)
grid = dict(
    degree=[1,2,3,4,5,6,7,8,9,10],
    gamma=['scale','auto'],
    C=uniform(loc=0, scale=8),
    coef0=uniform(loc=0, scale=8),
    max_iter=[1,25,50,100,500,1000,10000,100000,1000000],
    )
random_search = RandomizedSearchCV(rgs, grid, random_state=0)
search = random_search.fit(X_train, y_train)
search.best_params_

{'C': 7.134184006256638,
 'coef0': 7.709302084008234,
 'degree': 2,
 'gamma': 'scale',
 'max_iter': 100000}

In [16]:
rgs = svm.SVR(kernel='poly',cache_size=1024, C=7.134184006256638, coef0=7.709302084008234, degree=2,gamma='scale', max_iter=100000)
pipe_scaler_regressor = Pipeline(
[
    ('scaler', scaler),
    ('regressor', rgs)
])

pipe_scaler_regressor.fit(X_train,y_train)
y_pred = pipe_scaler_regressor.predict(X_test)
print('RMSE : ' + str(mean_squared_error(y_test, y_pred, squared=False)))
print('R2 squared : ' + str(r2_score(y_test,y_pred)))
print('Mean of the target variable: ' + str(y.mean()))

RMSE : 1.711955212964342
R2 squared : 0.9578395327794252
Mean of the target variable: 7.473286999999999


# Refleksja na temat skalowania

Empirycznie zostało sprawdzone, że dla użytych przez mnie ramech danych skalowanie znacznie poprawia pracę modelu. Tak jak było powiedziane w atykule, svm jest wrażliwy na skalowanie i znacznie wpływa na wyniki. Jako że SVM próbuje zmaksymalizować odległość między płaszczyzną rozdzielającą a wektorami nośnymi toeśli jedna cecha ma bardzo duże wartości to będzie dominować nad innymi cechami podczas obliczania odległości. W powyższych danych tak właśnie jest, niektóre kolumny zawierają wartości duże większe od reszty (np rok zbudowania budynku, lub pensja piłkarzy). Po przeskalowaniu wszystkich obiektów wszystkie mają taki sam wpływ